# CPSC4810 Team Project 
#### Leo Liang
#### Qirui Cao
#### Latesh Subramanayam

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

## Section 1 Data Inspection and Cleaning

(description data sources for each dataset:)

### Inspect the county data set

In [20]:
dfCounty = pd.read_csv("data/us_county.csv")

In [21]:
dfCounty.head(5)

,fips,county,state,state_code,male,female,median_age,population,female_percentage,lat,long
0,1001,Autauga County,Alabama,AL,26874,28326,37.8,55200,51.315217,32.534923,-86.642730
1,1003,Baldwin County,Alabama,AL,101188,106919,42.8,208107,51.376936,30.727479,-87.722564
2,1005,Barbour County,Alabama,AL,13697,12085,39.9,25782,46.873788,31.869581,-85.393210
3,1007,Bibb County,Alabama,AL,12152,10375,39.9,22527,46.055844,32.998628,-87.126475
4,1009,Blount County,Alabama,AL,28434,29211,40.8,57645,50.673953,33.980869,-86.567380


In [22]:
print("County dataset has", dfCounty.shape[0], "rows and", dfCounty.shape[1], "columns in the dataset.")

County dataset has 3220 rows and 11 columns in the dataset.


In [23]:
print("The number of rows in the dataset is equal to the number of unique values in column 'fips':", 
      len(dfCounty) == dfCounty.fips.nunique())

The number of rows in the dataset is equal to the number of unique values in column 'fips': True


Since the number of rows of the dataframe is equal to the number of unique values for variable "fips". We can consider the column "fips" as the key of the dataframe. That is, each row in dataframe dfCounty represents a different US county. 

In [24]:
dfCounty.isnull().sum()

fips                  0
county                0
state                 0
state_code           79
male                  0
female                0
median_age            0
population            0
female_percentage     0
lat                   0
long                  0
dtype: int64

Then, by running the total number of missing values for each variable, we find out that there are 79 missing values for "state_code". Then we need to check those rows containing missing values. 

In [25]:
dfCounty[dfCounty.state_code.isnull()]['state'].value_counts()

Puerto Rico             78
District of Columbia     1
Name: state, dtype: int64

In [26]:
print("Puerto Rico has", len(dfCounty[dfCounty.state == "Puerto Rico"]), "rows in the dataframe. ")

Puerto Rico has 78 rows in the dataframe. 


In [27]:
print("District of Columbia", len(dfCounty[dfCounty.state == "District of Columbia"]), "rows in the dataframe")

District of Columbia 1 rows in the dataframe


We find out that the only two states which do not have a state code are Puerto Rico and District of Columbia. By comparing the number of rows for two states, and the number of missing value occurance, we can be sure that state Puerto Rico and District of Columbia do not have state code. 

This is because both areas are not a US state, then they do not have a US state code. Since Puerto Rico is not part of 50 states + 1 disctrict, we exclude those observations from the project. 

In [31]:
dfCounty = dfCounty.drop(dfCounty[dfCounty.state == "Puerto Rico"].index)
dfCounty.state.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [32]:
dfCounty.describe()

,fips,male,female,median_age,population,female_percentage,lat,long
count,3142.000000,3.142000e+03,3.142000e+03,3142.000000,3.142000e+03,3142.000000,3142.000000,3142.000000
mean,30383.649268,5.059968e+04,5.217022e+04,41.288192,1.027699e+05,49.913803,38.455996,-92.267500
std,15162.508374,1.620390e+05,1.679069e+05,5.413554,3.299077e+05,2.379820,5.308022,12.860048
min,1001.000000,3.800000e+01,3.700000e+01,21.700000,7.500000e+01,21.003945,19.598721,-164.027212
25%,18177.500000,5.495750e+03,5.461250e+03,38.000000,1.094800e+04,49.395101,34.694147,-98.232470
50%,29176.000000,1.284900e+04,1.284850e+04,41.200000,2.573600e+04,50.378494,38.379236,-90.400299
75%,45080.500000,3.360025e+04,3.416750e+04,44.400000,6.720900e+04,51.107509,41.807375,-83.439403
max,56045.000000,4.976788e+06,5.121264e+06,67.000000,1.009805e+07,58.613660,69.312150,-67.628790


Then, we aggregate the population by all states to replicate the raw dataframe with state-level information.  

In [33]:
state_pop = pd.DataFrame(dfCounty.groupby('state')[['male','female','population']].sum())
state_pop["female_proportion"] = state_pop.female/state_pop.population
state_pop.reset_index(level=0, inplace=True)
state_pop

,state,male,female,population,female_proportion
0,Alabama,2355799,2508881,4864680,0.515734
1,Alaska,385579,352937,738516,0.477900
2,Arizona,3453439,3493246,6946685,0.502865
3,Arkansas,1468412,1522259,2990671,0.509002
4,California,19453769,19694991,39148760,0.503081
5,Colorado,2780831,2750310,5531141,0.497241
6,Connecticut,1747131,1834373,3581504,0.512180
7,Delaware,459570,489925,949495,0.515985
8,District of Columbia,324881,359617,684498,0.525373
9,Florida,10071925,10526214,20598139,0.511027


### Inspect the Poverty data 

In [137]:
dfPoverty = pd.read_csv("data/PovertyEstimates_us_county.csv")

In [138]:
dfPoverty.head(5)

,FIPStxt,Stabr,Area_name,Rural-urban_Continuum_Code_2003,Urban_Influence_Code_2003,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_2013,POVALL_2018,CI90LBAll_2018,CI90UBALL_2018,...,CI90UB517P_2018,MEDHHINC_2018,CI90LBINC_2018,CI90UBINC_2018,POV04_2018,CI90LB04_2018,CI90UB04_2018,PCTPOV04_2018,CI90LB04P_2018,CI90UB04P_2018
0,0,US,United States,NaN,NaN,NaN,NaN,41852315,41619366,42085264,...,17.2,61937,61843,62031,3758704.0,3714862.0,3802546.0,19.5,19.3,19.7
1,1000,AL,Alabama,NaN,NaN,NaN,NaN,801758,785668,817848,...,23.7,49881,49123,50639,73915.0,69990.0,77840.0,26.0,24.6,27.4
2,1001,AL,Autauga County,2.0,2.0,2.0,2.0,7587,6334,8840,...,23.9,59338,53628,65048,NaN,NaN,NaN,NaN,NaN,NaN
3,1003,AL,Baldwin County,4.0,5.0,3.0,2.0,21069,17390,24748,...,16.9,57588,54437,60739,NaN,NaN,NaN,NaN,NaN,NaN
4,1005,AL,Barbour County,6.0,6.0,6.0,6.0,6788,5662,7914,...,45.9,34382,31157,37607,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
dfPoverty.columns

Index(['FIPStxt', 'Stabr', 'Area_name', 'Rural-urban_Continuum_Code_2003',
       'Urban_Influence_Code_2003', 'Rural-urban_Continuum_Code_2013',
       'Urban_Influence_Code_2013', 'POVALL_2018', 'CI90LBAll_2018',
       'CI90UBALL_2018', 'PCTPOVALL_2018', 'CI90LBALLP_2018',
       'CI90UBALLP_2018', 'POV017_2018', 'CI90LB017_2018', 'CI90UB017_2018',
       'PCTPOV017_2018', 'CI90LB017P_2018', 'CI90UB017P_2018', 'POV517_2018',
       'CI90LB517_2018', 'CI90UB517_2018', 'PCTPOV517_2018', 'CI90LB517P_2018',
       'CI90UB517P_2018', 'MEDHHINC_2018', 'CI90LBINC_2018', 'CI90UBINC_2018',
       'POV04_2018', 'CI90LB04_2018', 'CI90UB04_2018', 'PCTPOV04_2018',
       'CI90LB04P_2018', 'CI90UB04P_2018'],
      dtype='object')

Then, we altered the dataset to keep the relevant columns.

In [140]:
dfPoverty = dfPoverty[['FIPStxt', 'Stabr', 'Area_name', 'Rural-urban_Continuum_Code_2013', 'Urban_Influence_Code_2013', 
                        'POVALL_2018', 'PCTPOVALL_2018', 'POV017_2018', 'PCTPOV017_2018', 'MEDHHINC_2018']]

In [141]:
print("Poverty dataset has", dfPoverty.shape[0], "rows and", dfPoverty.shape[1], "columns in the dataset.")

Poverty dataset has 3193 rows and 10 columns in the dataset.


In [142]:
print("The number of rows in the dataset is equal to the number of unique values in column 'fips':", 
      len(dfPoverty) == dfPoverty["FIPStxt"].nunique())

The number of rows in the dataset is equal to the number of unique values in column 'fips': True


Similarly, since the number of rows of the dataframe is equal to the number of unique values for variable "FIPStxt". We can consider the column "FIPStxt" as the key of the dataframe. That is, each row in dataframe dfCounty represents a different US area. 

In [143]:
dfPoverty.isnull().sum()

FIPStxt                             0
Stabr                               0
Area_name                           0
Rural-urban_Continuum_Code_2013    52
Urban_Influence_Code_2013          52
POVALL_2018                         0
PCTPOVALL_2018                      0
POV017_2018                         0
PCTPOV017_2018                      0
MEDHHINC_2018                       0
dtype: int64

Then, by running the total number of missing values for each variable, we find out that there are 52 missing values for 2013's "RUCC" and "UIC". Then we need to check those rows containing missing values.

In [144]:
dfPoverty[dfPoverty["Rural-urban_Continuum_Code_2013"].isna()]["Area_name"]

0              United States
1                    Alabama
69                    Alaska
99                   Arizona
115                 Arkansas
191               California
250                 Colorado
315              Connecticut
324                 Delaware
328     District of Columbia
330                  Florida
398                  Georgia
558                   Hawaii
563                    Idaho
608                 Illinois
711                  Indiana
804                     Iowa
904                   Kansas
1010                Kentucky
1131               Louisiana
1196                   Maine
1213                Maryland
1238           Massachusetts
1253                Michigan
1337               Minnesota
1425             Mississippi
1508                Missouri
1624                 Montana
1681                Nebraska
1775                  Nevada
1793           New Hampshire
1804              New Jersey
1826              New Mexico
1860                New York
1923          

In [145]:
dfPoverty[dfPoverty["Urban_Influence_Code_2013"].isna()]["Area_name"]

0              United States
1                    Alabama
69                    Alaska
99                   Arizona
115                 Arkansas
191               California
250                 Colorado
315              Connecticut
324                 Delaware
328     District of Columbia
330                  Florida
398                  Georgia
558                   Hawaii
563                    Idaho
608                 Illinois
711                  Indiana
804                     Iowa
904                   Kansas
1010                Kentucky
1131               Louisiana
1196                   Maine
1213                Maryland
1238           Massachusetts
1253                Michigan
1337               Minnesota
1425             Mississippi
1508                Missouri
1624                 Montana
1681                Nebraska
1775                  Nevada
1793           New Hampshire
1804              New Jersey
1826              New Mexico
1860                New York
1923          

By observing the missing values in both columns, the corresponding area names are states and US itself. The missing values are due to the fact that the country US itself and states are not applied to the county-level code, as it only applies to counties. 

Meanwhile, Rural-urban Continuum Code 2013 and Urban Influence Code 2013 are all categorical variables, which have their corresponding categories. (Please see documents below) Then, we need to map the code to their corresponding description. 

Rural-Urban Continuum Code: https://www.ers.usda.gov/data-products/rural-urban-continuum-codes/documentation/
Urban Influence Code: https://www.ers.usda.gov/data-products/urban-influence-codes/documentation.aspx.

In [146]:
RUCC = {1: "Counties in metro areas of 1 million population or more", 
           2: "Counties in metro areas of 250,000 to 1 million population",
           3: "Counties in metro areas of fewer than 250,000 population",
           4: "Urban population of 20,000 or more, adjacent to a metro area",
           5: "Urban population of 20,000 or more, not adjacent to a metro area",
           6: "Urban population of 2,500 to 19,999, adjacent to a metro area", 
           7: "Urban population of 2,500 to 19,999, not adjacent to a metro area", 
           8: "Completely rural or less than 2,500 urban population, adjacent to a metro area",
           9: "Completely rural or less than 2,500 urban population, not adjacent to a metro area"
          }

UIC = {1: "In large metro area of 1+ million residents",
       2: "In small metro area of less than 1 million residents",
       3: "Micropolitan area adjacent to large metro area",
       4: "Noncore adjacent to large metro area",
       5: "Micropolitan area adjacent to small metro area",
       6: "Noncore adjacent to small metro area and contains a town of at least 2,500 residents",
       7: "Noncore adjacent to small metro area and does not contain a town of at least 2,500 residents",
       8: "Micropolitan area not adjacent to a metro area",
       9: "Noncore adjacent to micro area and contains a town of at least 2,500 residents",
       10: "Noncore adjacent to micro area and does not contain a town of at least 2,500 residents",
       11: "Noncore not adjacent to metro or micro area and contains a town of at least 2,500 residents",
       12: "Noncore not adjacent to metro or micro area and does not contain a town of at least 2,500 residents"
      }

dfPoverty["Rural-urban_Continuum_Code_2013"] = dfPoverty["Rural-urban_Continuum_Code_2013"].map(RUCC)
dfPoverty["Urban_Influence_Code_2013"] = dfPoverty["Urban_Influence_Code_2013"].map(UIC)

In [147]:
dfPoverty.head(5)

,FIPStxt,Stabr,Area_name,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_2013,POVALL_2018,PCTPOVALL_2018,POV017_2018,PCTPOV017_2018,MEDHHINC_2018
0,0,US,United States,NaN,NaN,41852315,13.1,12997532,18.0,61937
1,1000,AL,Alabama,NaN,NaN,801758,16.8,255613,23.9,49881
2,1001,AL,Autauga County,"Counties in metro areas of 250,000 to 1 millio...",In small metro area of less than 1 million res...,7587,13.8,2509,19.3,59338
3,1003,AL,Baldwin County,"Counties in metro areas of fewer than 250,000 ...",In small metro area of less than 1 million res...,21069,9.8,6442,13.9,57588
4,1005,AL,Barbour County,"Urban population of 2,500 to 19,999, adjacent ...",Noncore adjacent to small metro area and conta...,6788,30.9,2242,43.9,34382


### Inspect COVID case and death data

In [148]:
dfcovCnty = pd.read_csv("data/covid_us_county.csv")

In [149]:
dfcovCnty.head(5)

,fips,county,state,lat,long,date,cases,state_code,deaths
0,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-22,0,AL,0
1,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-23,0,AL,0
2,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-24,0,AL,0
3,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-25,0,AL,0
4,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-26,0,AL,0


In [150]:
print("COVID dataset has", dfcovCnty.shape[0], "rows and", dfcovCnty.shape[1], "columns in the dataset.")

COVID dataset has 1858152 rows and 9 columns in the dataset.


This data contains the county-level cumulative cases and deaths starting from January 22nd, 2020. Let's take the current date, and inspect the missing values

In [151]:
dfcovCnty_current = dfcovCnty[dfcovCnty['date'] == '2021-07-30']
dfcovCnty_current.head()

,fips,county,state,lat,long,date,cases,state_code,deaths
555,1001.0,Autauga,Alabama,32.539527,-86.644082,2021-07-30,7479,AL,114
1111,1003.0,Baldwin,Alabama,30.727750,-87.722071,2021-07-30,24499,AL,328
1667,1005.0,Barbour,Alabama,31.868263,-85.387129,2021-07-30,2462,AL,61
2223,1007.0,Bibb,Alabama,32.996421,-87.125115,2021-07-30,2845,AL,65
2779,1009.0,Blount,Alabama,33.982109,-86.567906,2021-07-30,7266,AL,139


In [152]:
print("current date COVID dataset has", dfcovCnty_current.shape[0], "rows and", dfcovCnty_current.shape[1], "columns in the dataset.")

current date COVID dataset has 3342 rows and 9 columns in the dataset.


In [153]:
dfcovCnty_current.isnull().sum()

fips          10
county         6
state          0
lat            0
long           0
date           0
cases          0
state_code    89
deaths         0
dtype: int64

There are 10 missing values in column "fips", 6 in column "county" and 89 in column "state_code". We need to check all the rows containing these missing values. 

In [154]:
dfcovCnty_current[dfcovCnty_current.fips.isnull()]

,fips,county,state,lat,long,date,cases,state_code,deaths
706119,NaN,Dukes and Nantucket,Massachusetts,41.406747,-70.687635,2021-07-30,3009,MA,6
726691,NaN,Federal Correctional Institution (FCI),Michigan,0.000000,0.000000,2021-07-30,320,MI,5
744483,NaN,Michigan Department of Corrections (MDOC),Michigan,0.000000,0.000000,2021-07-30,26770,MI,155
886263,NaN,Kansas City,Missouri,39.099700,-94.578600,2021-07-30,49320,MO,617
1644091,NaN,Bear River,Utah,41.521068,-113.083282,2021-07-30,23219,UT,114
1646871,NaN,Central Utah,Utah,39.372319,-111.575868,2021-07-30,9650,UT,72
1657435,NaN,Southeast Utah,Utah,38.996171,-110.701396,2021-07-30,3975,UT,31
1657991,NaN,Southwest Utah,Utah,37.854472,-111.441876,2021-07-30,31376,UT,289
1659659,NaN,TriCounty,Utah,40.124915,-109.517442,2021-07-30,5053,UT,42
1664107,NaN,Weber-Morgan,Utah,41.271160,-111.914512,2021-07-30,33602,UT,233


Since in the merging stage of our project, we will merge by fips, which is the key column for most of our county-level dataframe, we will remove those rows containing missing values in fips. 

In [155]:
dfcovCnty_current[dfcovCnty_current.county.isnull()]

,fips,county,state,lat,long,date,cases,state_code,deaths
57267,60.0,NaN,American Samoa,-14.2710,-170.1320,2021-07-30,0,NaN,0
188483,88888.0,NaN,Diamond Princess,0.0000,0.0000,2021-07-30,49,NaN,0
318587,99999.0,NaN,Grand Princess,0.0000,0.0000,2021-07-30,103,NaN,3
319143,66.0,NaN,Guam,13.4443,144.7937,2021-07-30,8541,NaN,143
1180943,69.0,NaN,Northern Mariana Islands,15.0979,145.6739,2021-07-30,183,NaN,2
1673559,78.0,NaN,Virgin Islands,18.3358,-64.8963,2021-07-30,4606,NaN,37


Since the missing values in county indicates two cruise ships and four areas outside 50 states + 1 district, we remove those rows as well. 

In [156]:
dfcovCnty=dfcovCnty.dropna(subset=['fips','county'])
dfcovCnty_current=dfcovCnty_current.dropna(subset=['fips','county'])

Then, we checked the rows with missing state codes, and found out that only Puerto Rico and District of Columbia don't have state code. It is the same case as the previous situation where both areas are not a US state, thus there are no state codes. And we removed rows with Puerto Rico as well.

In [157]:
dfcovCnty_current[dfcovCnty_current.state_code.isnull()].state.value_counts()

Puerto Rico             80
District of Columbia     3
Name: state, dtype: int64

In [158]:
print("Puerto Rico has", len(dfcovCnty_current[dfcovCnty_current.state == "Puerto Rico"]), "rows in the dataframe. ")

Puerto Rico has 80 rows in the dataframe. 


In [159]:
print("District of Columbia", len(dfcovCnty_current[dfcovCnty_current.state == "District of Columbia"]), "rows in the dataframe")

District of Columbia 3 rows in the dataframe


In [160]:
dfcovCnty = dfcovCnty.drop(dfcovCnty[dfcovCnty.state == "Puerto Rico"].index)
dfcovCnty_current = dfcovCnty[dfcovCnty.date == '2021-07-30']
dfcovCnty.state.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [161]:
print("Now, the number of rows in the dataset is equal to the number of unique values in column 'fips':", 
      len(dfcovCnty_current) == dfcovCnty_current.fips.nunique())

Now, the number of rows in the dataset is equal to the number of unique values in column 'fips': True


Since the number of rows of the dataframe is equal to the number of unique values for variable "fips". We can consider the column "fips" as the key of the dataframe. That is, each row in dataframe current represents a different US county.

In [162]:
dfcovCnty_current.describe()

,fips,lat,long,cases,deaths
count,3246.000000,3246.000000,3246.000000,3.246000e+03,3246.000000
mean,32083.336106,37.260840,-89.428180,1.065934e+04,187.518792
std,17742.211826,8.522405,20.596120,3.834142e+04,723.357640
min,1001.000000,0.000000,-174.159600,0.000000e+00,0.000000
25%,19041.500000,34.263872,-98.062361,1.020000e+03,18.000000
50%,30046.000000,38.162993,-89.900575,2.568500e+03,49.000000
75%,47008.500000,41.685722,-82.841249,6.731750e+03,117.000000
max,90056.000000,69.314792,0.000000,1.297087e+06,24691.000000


Similar to the county data, we also aggregated the cumulative cases and deaths by state level to replicate the dataframe. 

In [163]:
dfcovSt = pd.DataFrame(dfcovCnty.groupby(["state", "date"])[["cases", "deaths"]].sum())
dfcovSt.reset_index(inplace=True)
dfcovSt.head()

,state,date,cases,deaths
0,Alabama,2020-01-22,0,0
1,Alabama,2020-01-23,0,0
2,Alabama,2020-01-24,0,0
3,Alabama,2020-01-25,0,0
4,Alabama,2020-01-26,0,0


In [164]:
dfcovSt_current = dfcovSt[dfcovSt.date == "2021-07-30"]
dfcovSt_current.head()

,state,date,cases,deaths
555,Alabama,2021-07-30,577463,11510
1111,Alaska,2021-07-30,75486,389
1667,Arizona,2021-07-30,925169,18224
2223,Arkansas,2021-07-30,385113,6123
2779,California,2021-07-30,3956303,64423


### Inspect Vaccination Data

In [165]:
dfvac = pd.read_csv("data/us_state_vaccinations.csv")

In [166]:
dfvac.head()

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,2021-01-12,Alabama,78134.0,377025.0,70861.0,0.15,1.59,7270.0,1.45,7.69,NaN,NaN,NaN,0.207
1,2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.53,7.73,5906.0,5906.0,1205.0,0.222
2,2021-01-14,Alabama,92300.0,435350.0,80480.0,NaN,1.88,NaN,1.64,8.88,8260.0,7083.0,1445.0,0.212
3,2021-01-15,Alabama,100567.0,444650.0,86956.0,0.28,2.05,13488.0,1.77,9.07,8267.0,7478.0,1525.0,0.226
4,2021-01-16,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7557.0,7498.0,1529.0,NaN


In [167]:
print("Vaccine dataset has", dfvac.shape[0], "rows and", dfvac.shape[1], "columns in the dataset.")

Vaccine dataset has 13088 rows and 14 columns in the dataset.


Since the dataset contains the time series state level data of US vaccination, we subsetted the current date as an example to examine the data.

In [168]:
dfvac_current = dfvac[dfvaccine.date == '2021-07-30']
dfvac_current.head()

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
199,2021-07-30,Alabama,3586810.0,5092510.0,2116551.0,34.33,73.15,1683367.0,43.17,103.86,28844.0,13131.0,2678.0,0.704
400,2021-07-30,Alaska,696377.0,853805.0,376154.0,45.48,95.19,332678.0,51.42,116.71,1214.0,837.0,1144.0,0.816
601,2021-07-30,American Samoa,51670.0,55830.0,28768.0,42.31,92.78,23560.0,51.66,100.25,0.0,229.0,4112.0,0.925
802,2021-07-30,Arizona,7085856.0,8540830.0,3853257.0,45.19,97.35,3289110.0,52.94,117.34,17540.0,12063.0,1657.0,0.830
1003,2021-07-30,Arkansas,2432736.0,3004150.0,1405459.0,36.29,80.61,1095214.0,46.57,99.55,10935.0,9842.0,3261.0,0.810


In [169]:
dfvac_current.isnull().sum()

date                                   0
location                               0
total_vaccinations                     0
total_distributed                      1
people_vaccinated                      1
people_fully_vaccinated_per_hundred    4
total_vaccinations_per_hundred         4
people_fully_vaccinated                1
people_vaccinated_per_hundred          4
distributed_per_hundred                4
daily_vaccinations_raw                 0
daily_vaccinations                     0
daily_vaccinations_per_million         4
share_doses_used                       1
dtype: int64

Looking at the unique locations, we will only be keeping the states and the capital city

In [170]:
dfvac_current.location.unique()

array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'Bureau of Prisons', 'California', 'Colorado', 'Connecticut',
       'Delaware', 'Dept of Defense', 'District of Columbia',
       'Federated States of Micronesia', 'Florida', 'Georgia', 'Guam',
       'Hawaii', 'Idaho', 'Illinois', 'Indian Health Svc', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Long Term Care', 'Louisiana',
       'Maine', 'Marshall Islands', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York State', 'North Carolina', 'North Dakota',
       'Northern Mariana Islands', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Puerto Rico', 'Republic of Palau', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas',
       'United States', 'Utah', 'Vermont', 'Veterans Health',
       'Virgin Islands', 'Virginia', 'Washington', 'West V

Only location where it is a state will be kept.

In [171]:
dfvac_current = dfvac_current[~dfvac_current.location.isin(['American Samoa','Bureau of Prisons','Dept of Defense','Federated States of Micronesia',
               'Indian Health Svc','Long Term Care','Marshall Islands','Northern Mariana Islands',
               'Puerto Rico', 'Republic of Palau','United States','Veterans Health','Guam','Virgin Islands'])]
dfvac_current.location.nunique()

51

Similarly, we removed those locations from the time series dataframe as well. 

In [172]:
dfvac = dfvac[~dfvac.location.isin(['American Samoa','Bureau of Prisons','Dept of Defense','Federated States of Micronesia',
               'Indian Health Svc','Long Term Care','Marshall Islands','Northern Mariana Islands',
               'Puerto Rico', 'Republic of Palau','United States','Veterans Health','Guam','Virgin Islands'])]
dfvac.location.nunique()

51

checking for missing values again. We do not have missing values anymore.

In [173]:
dfvac_current.isnull().sum()

date                                   0
location                               0
total_vaccinations                     0
total_distributed                      0
people_vaccinated                      0
people_fully_vaccinated_per_hundred    0
total_vaccinations_per_hundred         0
people_fully_vaccinated                0
people_vaccinated_per_hundred          0
distributed_per_hundred                0
daily_vaccinations_raw                 0
daily_vaccinations                     0
daily_vaccinations_per_million         0
share_doses_used                       0
dtype: int64

now we have the 50 states and the capital city in the data

In [174]:
print("Now, the number of rows in the dataset is equal to the number of unique values in column 'location':", 
      len(dfvac_current) == dfvac_current.location.nunique())

Now, the number of rows in the dataset is equal to the number of unique values in column 'location': True


Since the number of rows of the dataframe is equal to the number of unique values for variable "location". We can consider the column "location" as the key column of the dataframe. That is, each row in dataframe current represents a different US county.

## Merge county-level data

Merging County info with covid cases by county (current only)

In [188]:
mergeCounty = pd.merge(dfCounty, dfcovCnty_current, how='inner', on='fips', validate = "1:1")
mergeCounty.head(5)

,fips,county_x,state_x,state_code_x,male,female,median_age,population,female_percentage,lat_x,long_x,county_y,state_y,lat_y,long_y,date,cases,state_code_y,deaths
0,1001,Autauga County,Alabama,AL,26874,28326,37.8,55200,51.315217,32.534923,-86.642730,Autauga,Alabama,32.539527,-86.644082,2021-07-30,7479,AL,114
1,1003,Baldwin County,Alabama,AL,101188,106919,42.8,208107,51.376936,30.727479,-87.722564,Baldwin,Alabama,30.727750,-87.722071,2021-07-30,24499,AL,328
2,1005,Barbour County,Alabama,AL,13697,12085,39.9,25782,46.873788,31.869581,-85.393210,Barbour,Alabama,31.868263,-85.387129,2021-07-30,2462,AL,61
3,1007,Bibb County,Alabama,AL,12152,10375,39.9,22527,46.055844,32.998628,-87.126475,Bibb,Alabama,32.996421,-87.125115,2021-07-30,2845,AL,65
4,1009,Blount County,Alabama,AL,28434,29211,40.8,57645,50.673953,33.980869,-86.567380,Blount,Alabama,33.982109,-86.567906,2021-07-30,7266,AL,139


In [189]:
mergeCounty.shape

(3142, 19)

Merge the above with poverty data

In [190]:
mergeCouPov = pd.merge(mergeCounty, dfPoverty, how='inner', left_on='fips', right_on='FIPStxt', validate = "1:1")
mergeCouPov

,fips,county_x,state_x,state_code_x,male,female,median_age,population,female_percentage,lat_x,...,FIPStxt,Stabr,Area_name,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_2013,POVALL_2018,PCTPOVALL_2018,POV017_2018,PCTPOV017_2018,MEDHHINC_2018
0,1001,Autauga County,Alabama,AL,26874,28326,37.8,55200,51.315217,32.534923,...,1001,AL,Autauga County,"Counties in metro areas of 250,000 to 1 millio...",In small metro area of less than 1 million res...,7587,13.8,2509,19.3,59338
1,1003,Baldwin County,Alabama,AL,101188,106919,42.8,208107,51.376936,30.727479,...,1003,AL,Baldwin County,"Counties in metro areas of fewer than 250,000 ...",In small metro area of less than 1 million res...,21069,9.8,6442,13.9,57588
2,1005,Barbour County,Alabama,AL,13697,12085,39.9,25782,46.873788,31.869581,...,1005,AL,Barbour County,"Urban population of 2,500 to 19,999, adjacent ...",Noncore adjacent to small metro area and conta...,6788,30.9,2242,43.9,34382
3,1007,Bibb County,Alabama,AL,12152,10375,39.9,22527,46.055844,32.998628,...,1007,AL,Bibb County,Counties in metro areas of 1 million populatio...,In large metro area of 1+ million residents,4400,21.8,1238,27.8,46064
4,1009,Blount County,Alabama,AL,28434,29211,40.8,57645,50.673953,33.980869,...,1009,AL,Blount County,Counties in metro areas of 1 million populatio...,In large metro area of 1+ million residents,7527,13.2,2374,18.0,50412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,56037,Sweetwater County,Wyoming,WY,22882,21235,34.6,44117,48.133373,41.659534,...,56037,WY,Sweetwater County,"Urban population of 20,000 or more, not adjace...",Micropolitan area not adjacent to a metro area,3540,8.4,1099,10.0,73315
3137,56039,Teton County,Wyoming,WY,11911,11148,39.3,23059,48.345548,43.934651,...,56039,WY,Teton County,"Urban population of 2,500 to 19,999, not adjac...",Micropolitan area not adjacent to a metro area,1443,6.3,278,6.7,99087
3138,56041,Uinta County,Wyoming,WY,10505,10104,35.5,20609,49.027124,41.287642,...,56041,WY,Uinta County,"Urban population of 2,500 to 19,999, not adjac...",Micropolitan area not adjacent to a metro area,2010,10.0,696,12.1,63401
3139,56043,Washakie County,Wyoming,WY,4137,3992,43.5,8129,49.108131,43.904970,...,56043,WY,Washakie County,"Urban population of 2,500 to 19,999, not adjac...",Noncore not adjacent to metro or micro area an...,918,11.9,277,16.1,55190


In [191]:
mergeCouPov.columns

Index(['fips', 'county_x', 'state_x', 'state_code_x', 'male', 'female',
       'median_age', 'population', 'female_percentage', 'lat_x', 'long_x',
       'county_y', 'state_y', 'lat_y', 'long_y', 'date', 'cases',
       'state_code_y', 'deaths', 'FIPStxt', 'Stabr', 'Area_name',
       'Rural-urban_Continuum_Code_2013', 'Urban_Influence_Code_2013',
       'POVALL_2018', 'PCTPOVALL_2018', 'POV017_2018', 'PCTPOV017_2018',
       'MEDHHINC_2018'],
      dtype='object')

In [193]:
mergeCouPov[mergeCouPov.county_x != mergeCouPov.Area_name]

,fips,county_x,state_x,state_code_x,male,female,median_age,population,female_percentage,lat_x,...,FIPStxt,Stabr,Area_name,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_2013,POVALL_2018,PCTPOVALL_2018,POV017_2018,PCTPOV017_2018,MEDHHINC_2018
69,2020,Anchorage Municipality,Alaska,AK,151144,144968,33.2,296112,48.957151,61.150825,...,2020,AK,Anchorage Borough,"Counties in metro areas of 250,000 to 1 millio...",In small metro area of less than 1 million res...,27075,9.5,8467,12.1,82113
77,2110,Juneau City and Borough,Alaska,AK,16517,15813,38.1,32330,48.911228,58.454631,...,2110,AK,Juneau Borough,"Urban population of 20,000 or more, not adjace...",Micropolitan area not adjacent to a metro area,2299,7.3,573,8.5,79173
89,2220,Sitka City and Borough,Alaska,AK,4471,4267,38.8,8738,48.832685,57.240077,...,2220,AK,Sitka Borough,"Urban population of 2,500 to 19,999, not adjac...",Noncore not adjacent to metro or micro area an...,656,7.7,159,8.8,71825
94,2282,Yakutat City and Borough,Alaska,AK,369,320,36.4,689,46.444122,59.888895,...,2282,AK,Yakutat Borough,"Completely rural or less than 2,500 urban popu...",Noncore not adjacent to metro or micro area an...,79,13.1,17,15.3,59523
643,17099,LaSalle County,Illinois,IL,55400,55001,41.8,110401,49.819295,41.343995,...,17099,IL,La Salle County,"Urban population of 20,000 or more, adjacent t...",Micropolitan area adjacent to large metro area,14384,13.6,4665,20.2,54063
712,18033,DeKalb County,Indiana,IN,21307,21397,39.1,42704,50.105377,41.397570,...,18033,IN,De Kalb County,"Urban population of 20,000 or more, adjacent t...",Micropolitan area adjacent to small metro area,4309,10.1,1297,12.8,58987
739,18087,LaGrange County,Indiana,IN,19752,19190,31.4,38942,49.278414,41.642621,...,18087,IN,Lagrange County,"Urban population of 2,500 to 19,999, adjacent ...",Noncore adjacent to small metro area and conta...,3018,7.8,1406,11.2,67498
741,18091,LaPorte County,Indiana,IN,57068,53484,40.4,110552,48.379043,41.545988,...,18091,IN,La Porte County,"Counties in metro areas of fewer than 250,000 ...",In small metro area of less than 1 million res...,16212,15.7,5727,24.9,52095
1141,22059,LaSalle Parish,Louisiana,LA,7730,7219,38.0,14949,48.290856,31.676701,...,22059,LA,La Salle Parish,"Urban population of 2,500 to 19,999, adjacent ...",Noncore adjacent to small metro area and conta...,2659,19.5,820,24.2,45742
1800,35011,De Baca County,New Mexico,NM,1052,1008,37.0,2060,48.932039,34.342452,...,35011,NM,DeBaca County,"Completely rural or less than 2,500 urban popu...",Noncore not adjacent to metro or micro area an...,361,20.4,143,36.3,35087


## Merge state-level data

In [112]:
mergeCovState = pd.merge(dfcovSt, state_pop, how='left', on = 'state')
mergeCovState

,state,date,cases,deaths,male,female,population,female_proportion
0,Alabama,2020-01-22,0,0,2355799,2508881,4864680,0.515734
1,Alabama,2020-01-23,0,0,2355799,2508881,4864680,0.515734
2,Alabama,2020-01-24,0,0,2355799,2508881,4864680,0.515734
3,Alabama,2020-01-25,0,0,2355799,2508881,4864680,0.515734
4,Alabama,2020-01-26,0,0,2355799,2508881,4864680,0.515734
...,...,...,...,...,...,...,...,...
28907,Wyoming,2021-07-26,64451,766,296979,284857,581836,0.489583
28908,Wyoming,2021-07-27,64623,776,296979,284857,581836,0.489583
28909,Wyoming,2021-07-28,64623,776,296979,284857,581836,0.489583
28910,Wyoming,2021-07-29,64955,776,296979,284857,581836,0.489583
